# Importing libraries

In [4]:
# QDRANT
## Importar o client
from qdrant_client import QdrantClient
## Criar nossa coleção
from qdrant_client.http.models import Distance, VectorParams

# LANGCHAIN
## Importar Qdrant como vector store
from langchain.vectorstores import Qdrant
## Importar OpenAI embeddings
from langchain.embeddings.openai import OpenAIEmbeddings
## Função para auxiliar na quebra do texto em chunks
from langchain.text_splitter import CharacterTextSplitter
## Módulo para facilitar o uso de vector stores em QA (question answering)
from langchain.chains import RetrievalQA
## Importar LLM
from langchain.llms import OpenAI
from dotenv import load_dotenv
import os


# Init the client

In [5]:
client = QdrantClient(host="localhost", port=6333)

# Create collection

In [4]:
client.create_collection(
    collection_name="openai_collection",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

True

## Loading environment variables, create LangChain and Qdrant instances for retrieve data

In [6]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# Create a LangChain instance to interact with the OpenAI's models to generate embaddings for text
embeddings = OpenAIEmbeddings()

# Create a Qdrant instance to retrieve data
vectorstore = Qdrant(
        client=client,
        collection_name="openai_collection",
        embeddings=embeddings
    )

C:\Users\zkaua\AppData\Local\Temp\ipykernel_21832\2011570663.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
C:\Users\zkaua\AppData\Local\Temp\ipykernel_21832\2011570663.py:6: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  vectorstore = Qdrant(


# Text is splitted, converted to vectorized database and inserted in database

In [7]:
# Function to split large text into smaller chunks
def get_chunks(text):# Create an instance of CharacterTextSplitter, a class that splits text into chunks
    text_splitter = CharacterTextSplitter(
        separator="\n",        # Split text by newlines
        chunk_size=1000,       # Maximum size of each chunk (in characters)
        chunk_overlap=200,     # Number of overlapping characters between chunks
        length_function=len    # The function used to calculate text length (in this case, just the length in characters)
    )
    chunks = text_splitter.split_text(text)# Use the splitter to actually split the input text into chunks
    return chunks# Return the list of chunks (which is a list of smaller text segments)

with open("./base_dados.txt") as f:# Open a file named "base_dados.txt" and read its content
    raw_text = f.read()  # Read the entire content of the file into a single string

texts = get_chunks(raw_text)# Call the get_chunks function to split the raw text into smaller chunks

# Assuming 'vectorstore' is a pre-defined object (perhaps a Qdrant or other vector store),
# this line adds the chunks of text to the vector store for indexing or further processing
vectorstore.add_texts(texts)


['38d595974bf84d75916aa4c371f8d7e5',
 '90da9f67a3104f35b7e2d10395096374',
 '28bb3431e998419f9c9a13d350bda75a',
 'befd319aef1b445492eecc7e0339ab81',
 '317f62516dcc4e36ab77c4e7a27b6844']

## Create RetrievalQA chain to consult

In [7]:
# Create a RetrievalQA chain using a pre-defined LLM (OpenAI) and vectorstore retriever
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),  # Specify OpenAI's LLM (Language Model) to answer questions
    chain_type="stuff",  # Use "stuff" chain type, which retrieves relevant documents and passes them directly to the LLM
    retriever=vectorstore.as_retriever()  # Use the vectorstore as a retriever for fetching relevant documents based on embeddings
)

C:\Users\zkaua\AppData\Local\Temp\ipykernel_21832\3755277632.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm=OpenAI(),


In [8]:
query = "Quem avalia a solicitação de compra ?"
response = qa.run(query)

print(response)

C:\Users\zkaua\AppData\Local\Temp\ipykernel_21832\3389011717.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa.run(query)


 O departamento de Compras.


In [9]:
query = "Quantos por cento das solicitações são revisadas pelo comitê?"
response = qa.run(query)

print(response)

 10%


In [10]:
query = "Qual o nome da empresa?"
response = qa.run(query)

print(response)

 A empresa é chamada de "Company X".
